# 要約 
このJupyter Notebookでは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、人間の好みに基づいたモデルのトレーニングおよび評価を行うための準備と実行が取り扱われています。具体的には、異なる応答を提示するチャットボットのモデルを比較し、どちらが好まれるかを予測するための機械学習モデルの構築を目指しています。

### 取り組んでいる問題
Notebookは、ユーザープロンプトに対する2つの異なる応答の好みを判定する「報酬モデル」を構築するという課題に取り組んでいます。具体的には、トレーニングデータを処理し、必要な特徴量を抽出し、最終的にモデルのトレーニングを行う過程を示しています。

### 主な手法とライブラリ
1. **データ読み込みと前処理**:
   - Pandasを使用してトレーニングデータおよびテストデータをCSVファイルから読み込み、各応答に関する特徴を処理するカスタム関数が定義されています。
   - Hugging Faceの`datasets`ライブラリを利用してデータセットを管理します。

2. **モデルの選定と設定**:
   - Hugging FaceのTransformersライブラリから、Mistralという事前学習済みの因果言語モデルを使用し、量子化設定を行った上でモデルを読み込みます。
   - LoRA（Low-Rank Adaptation）技術を用いて、モデルにファインチューニングする設定が適用されます。

3. **トレーニング**:
   - `Trainer`クラスを拡張してカスタムトレーナーを定義し、損失計算にクラス重みを用いるなどの工夫がされています。
   - モデルのトレーニングは、PyTorchを使用し、勾配チェックポイント機能や、評価メトリクスを定義して行います。

4. **評価**:
   - トレーニング後、ユーザーが提示したプロンプトに基づいてモデルがどのように応答するかを生成し、その性能を評価します。

このNotebookは、データの前処理からモデルのトレーニングと評価に至る一連の流れを詳細に記述し、実践的な機械学習プロジェクトの進め方を示しています。使用するライブラリには、NumPy、Pandas、Transformers、Hugging FaceのDatasets、PyTorch、PEFT（Parameter-Efficient Fine-Tuning）などがあります。

---


# 用語概説 
以下は、そのJupyter Notebook内の機械学習および深層学習関連の専門用語の簡単な解説です。特に、初心者がつまずきやすいものに焦点を当てています。

1. **量子化 (Quantization)**:
   - モデルのサイズを削減し、推論速度を向上させるために、モデルの重みを低精度（例：32ビットから8ビットや4ビット）に変換するプロセスです。これにより、メモリ使用量や計算コストが減少します。

2. **ダブル量子化 (Double Quantization)**:
   - 通常の量子化に加え、さらに別の段階で量子化を行うことです。この手法は、さらなる圧縮を実現するために使用されます。

3. **bfloat16 (Brain Floating Point)**:
   - Googleが開発した浮動小数点数形式で、深層学習の計算でよく使用されます。従来の16ビット浮動小数点よりも大きな範囲を持ちながら、表現精度をある程度保持します。

4. **Gradient Checkpointing (勾配チェックポイント)**:
   - メモリを節約する手法で、順伝播中に計算された中間結果を保持せず、逆伝播中に必要な部分を再計算することでメモリ使用量を削減します。

5. **クラス重み (Class Weights)**:
   - 不均衡なデータセットにおいて、特定のクラスの影響を強調または抑えるために、損失関数に独自の重みを設定することです。これにより、少数クラスの学習を改善します。

6. **トレーニング可能なパラメータ (Trainable Parameters)**:
   - モデルのトレーニングによって更新されるパラメータのことを指します。これには、モデルの重みやバイアスが含まれます。

7. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - モデルの全体を微調整するのではなく、特定の部分のみを微調整する手法で、少ないパラメータで効果的なトレーニングを実現します。

8. **LoRA (Low-Rank Adaptation)**:
   - PEFTの一種で、モデルの重みを低ランクで近似する手法です。これにより計算コストを削減し、効率的なファインチューニングが可能となります。

9. **FSDP (Fully Sharded Data Parallel)**:
   - モデルのパラメータを完全に分割し、複数のデバイスで並行してトレーニングしたりチェックポイントを保存したりするための手法です。これにより、メモリ効率が向上します。

10. **Data Collator (データコレータ)**:
    - トレーニングデータをバッチ処理する際に、必要なフォーマットに整形するための関数やオブジェクトです。特に、パディングやマスキングを考慮する必要があります。

11. **ウォームアップステップ (Warmup Steps)**:
    - トレーニングの初期段階で、学習率を徐々に増加させるためのステップです。これにより、モデルが安定し、トレーニングの初期の不安定さを軽減します。

これらの用語は、特に初心者が実務体験がない場合に馴染みが薄い範囲で、深層学習モデルやトレーニングプロセスを理解する上で重要です。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# たとえば、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数用
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# たとえば、これを実行することで（実行をクリックするか、Shift + Enterを押すことで）入力ディレクトリ内のすべてのファイルをリスト表示できます

import os # オペレーティングシステムに関する機能を使用するためのモジュール
for dirname, _, filenames in os.walk('/kaggle/input'): # "/kaggle/input"ディレクトリ内のファイルやフォルダを再帰的に巡回します
    for filename in filenames: # ファイル名のリストをループします
        print(os.path.join(dirname, filename)) # 各ファイルのフルパスを出力します

# あなたは現在のディレクトリ（/kaggle/working/）に最大20GBを保存できます。これは「すべて保存して実行」機能を使用してバージョンを作成したときに出力として保持されます
# また、一時ファイルを/kaggle/temp/に書き込むこともできますが、現在のセッションの外では保存されません

In [ ]:
# !pip install cupy-cuda11x>=12.0.0  # GPUを利用したCuPyライブラリのインストール（CUDA 11.x以上）
# !pip install packaging>=22  # パッケージ管理用ライブラリのインストール（バージョン22以上）
# !pip install shapely>=2.0.1  # 幾何学的操作を行うためのShapelyライブラリのインストール（バージョン2.0.1以上）
# !pip install numpy<1.26  # 数値計算用のNumPyライブラリのインストール（バージョン1.26未満）
# !pip install scipy<1.12  # 科学計算用のSciPyライブラリのインストール（バージョン1.12未満）
!pip install transformers datasets accelerate peft  # Transformers、データセット、加速化、PEFTライブラリのインストール

In [ ]:
# このコマンドは、マシンごとに一度だけ実行すれば十分です
!pip install -q -U bitsandbytes  # bitsandbytesライブラリのインストール（省略モードで、アップグレードする）
# !pip install -q -U git+https://github.com/huggingface/transformers.git  # Hugging FaceのTransformersライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U git+https://github.com/huggingface/peft.git  # Hugging FaceのPEFTライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U git+https://github.com/huggingface/accelerate.git  # Hugging FaceのAccelerateライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U datasets scipy ipywidgets  # データセット、SciPy、およびipywidgetsライブラリのインストール（アップグレード）
!pip install datasets  # datasetsライブラリのインストール

In [ ]:
from datasets import load_dataset  # datasetsモジュールからload_dataset関数をインポートします

# 'csv'形式のデータセットを読み込みます。特に'test.csv'ファイルを指定しています。
train_dataset = load_dataset('csv', data_files='/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを含むCSVファイルを読み込み、train_datasetに格納します

In [ ]:
from huggingface_hub import notebook_login  # Hugging Face Hubからnotebook_login関数をインポートします

# Hugging Face Hubへのログインプロンプトを表示します
notebook_login()  # ユーザーがHugging Face Hubにログインできるようにします。これにより、モデルやデータセットにアクセスできるようになります。

In [ ]:
import torch  # PyTorchライブラリをインポートします
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # Hugging FaceのTransformersライブラリから必要なクラスをインポートします

base_model_id = "mistralai/Mistral-7B-v0.1"  # 使用する基本モデルのIDを指定します
bnb_config = BitsAndBytesConfig(  # 量子化に関する設定を作成します
    load_in_4bit=True,  # 4ビットでモデルを読み込む設定をします
    bnb_4bit_use_double_quant=True,  # ダブル量子化を使用する設定を行います
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを指定します（nf4を使用）
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算データ型をbfloat16に設定します
)

# モデルを事前学習済みの状態から読み込みます。量子化設定とデバイスマップを指定します。
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")  # 自動デバイスマッピングでモデルを読み込みます

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(  # 基本モデルのトークナイザーを読み込みます
    base_model_id,  # 使用するモデルIDを指定します
    model_max_length=512,  # モデルの最大長を512トークンに設定します
    padding_side="left",  # パディングを左側に設定します
    add_eos_token=True  # 終了トークンを追加する設定をします
)

# トークナイザーのパディングトークンを終了トークンに設定します
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンと同じに設定することで、モデル入力の整合性を保ちます

In [ ]:
# Pandasライブラリを使用して、CSVファイルを読み込みます
train_data = pd.read_csv('../input/lmsys-chatbot-arena/train.csv')  # トレーニングデータを読み込みます
test_data = pd.read_csv('../input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込みます
submission_data = pd.read_csv('../input/lmsys-chatbot-arena/sample_submission.csv')  # 提出用サンプルデータを読み込みます

In [ ]:
# 各データセットの形状（行数と列数）を表示します
train_data.shape, test_data.shape, submission_data.shape  # トレーニングデータ、テストデータ、および提出データのサイズを確認します

```python
def process(input_str):  # 入力文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 角括弧を削除します
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文を分離し、両端の引用符を削除します
    return  ' '.join(sentences)  # 文をスペースで結合して返します

def trim_endings(custom_string):  # 文字列の末尾をトリミングする関数
    return custom_string[:-2][2:]  # 最後の2文字を削除し、最初の2文字を除去して返します

def count_newlines(custom_string):  # 改行文字のカウントを行う関数
    return custom_string.count('\\n')  # 文字列内の改行数を返します

def word_counts(custom_string):  # 単語のカウントを行う関数
    return len(custom_string.split())  # 文字列を単語に分割し、その数を返します


def apply_transformations(df):  # データフレームに変換を適用する関数
    
    df['prompt'] =  df['prompt'].map(process)  # 'prompt'列にprocess関数を適用します
    df['response_a'] =  df['response_a'].map(process)  # 'response_a'列にprocess関数を適用します
    df['response_b'] =  df['response_b'].map(process)  # 'response_b'列にprocess関数を適用します
    
    df['prompt'] =  df['prompt'].map(trim_endings)  # 'prompt'列の末尾をトリミングします
    df['response_a'] =  df['response_a'].map(trim_endings)  # 'response_a'列の末尾をトリミングします
    df['response_b'] =  df['response_b'].map(trim_endings)  # 'response_b'列の末尾をトリミングします
    
    df['res_a_line_count'] = df['response_a'].map(count_newlines).astype(str)  # 'response_a'の改行数を計算し、新しい列に格納します
    df['res_b_line_count'] = df['response_b'].map(count_newlines).astype(str)  # 'response_b'の改行数を計算し、新しい列に格納します
    
    df['res_a_word_count'] = df['response_a'].map(word_counts).astype(str)  # 'response_a'の単語数を計算し、新しい列に格納します
    df['res_b_word_count'] = df['response_b'].map(word_counts).astype(str)  # 'response_b'の単語数を計算し、新しい列に格納します
    
    return df  # 変換後のデータフレームを返します

def create_summary(df):  # サマリーを作成する関数
    df['summary'] = 'User prompt: ' + df['prompt'] +  '\n\n Model A :\n' + df['response_a'] +'\n\n Model A length:\n' + df['res_a_word_count'] +'\n\n Model A Line Count:\n' + df['res_a_line_count'] +'\n\nModel B:\n'  + df['response_b'] +'\n\n \n\nModel B length:\n'  + df['res_b_word_count'] +'\n\n \n\nModel B Line Count:\n'  + df['res_b_line_count']
    return df  # データフレームにサマリー列を追加して返します
```

In [ ]:
# トレーニングデータに変換を適用し、サマリーを作成します
train_data = apply_transformations(train_data)  # トレーニングデータに変換を適用します
train_data = create_summary(train_data)  # トレーニングデータにサマリーを作成します

# テストデータにも同様に変換を適用し、サマリーを作成します
test_data = apply_transformations(test_data)  # テストデータに変換を適用します    
test_data = create_summary(test_data)  # テストデータにサマリーを作成します

In [ ]:
# トレーニングデータのサマリー列の3番目の要素を表示します
train_data['summary'][2]  # インデックス2のサマリーを取得して表示します

In [ ]:
# トレーニングデータにラベル列を追加します。最高スコアのモデルを特定します
train_data['labels'] = train_data[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)  # 各行で最大値を持つ列のインデックスを取得して'labels'列に格納します

train_data['labels'] = train_data['labels'].astype('category')  # 'labels'列をカテゴリ型に変換します
train_data['target'] = train_data['labels'].cat.codes  # 'labels'のカテゴリコードを' target'列に格納します

In [ ]:
# 'labels'列のカテゴリに対するコードを作成します
category_map = {code: category for code, category in enumerate(train_data['labels'].cat.categories)}  # 各カテゴリに対するコードを辞書形式で作成します
category_map  # カテゴリマップを表示します

In [ ]:
# 再現性を持たせるためのランダムシードを設定します
SEED = 7920  # シード値を7920に設定します。これにより、結果が一貫性を持つようになります

In [ ]:
from sklearn.datasets import make_classification  # データセットを生成するためのsklearnの関数をインポートします
from sklearn.model_selection import train_test_split  # データを訓練用と検証用に分割するための関数をインポートします
from sklearn.linear_model import LogisticRegression  # ロジスティック回帰モデルをインポートします
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score  # 評価指標を計算するための関数をインポートします

# 使用するカラムを選択してトレーニングデータとテストデータを準備します
train_df = train_data[['id', 'summary', 'target']]  # トレーニングデータから'id', 'summary', 'target'を選択します
test_df = train_data[['id', 'summary']]  # テストデータから'id'と'summary'を選択します

# トレーニングデータを訓練セットと検証セットに分割します
df_train, df_val = train_test_split(train_df, test_size=0.2, random_state=SEED, stratify=train_df['target'])  # 20%を検証データとして分割します。分類のバランスを保ちます

In [ ]:
from datasets import Dataset, DatasetDict  # datasetsモジュールからDatasetおよびDatasetDictをインポートします

# PandasデータフレームからHugging FaceのDataset形式に変換します
dataset_train = Dataset.from_pandas(df_train, preserve_index=False)  # トレーニングデータをDatasetに変換します（インデックスは保存しません）
dataset_val = Dataset.from_pandas(df_val, preserve_index=False)  # 検証データをDatasetに変換します（インデックスは保存しません）
dataset_test = Dataset.from_pandas(test_df, preserve_index=False)  # テストデータをDatasetに変換します（インデックスは保存しません）

In [ ]:
# トレーニングデータセットをシャッフルして、新しいデータセットを作成します
dataset_train_shuffled = dataset_train.shuffle(seed=SEED)  # 指定したシードを使用してデータをランダムに並び替えます

In [ ]:
# データセットを辞書形式でまとめて、トレーニング、検証、テストのデータセットを管理します
dataset = DatasetDict({  # データセットの辞書を作成します
    'train': dataset_train_shuffled,  # シャッフルしたトレーニングデータを追加します
    # 'val': dataset_val,  # 検証データを追加します（コメントアウト中）
    # 'test': dataset_test  # テストデータを追加します（コメントアウト中）
})
dataset  # 作成したデータセット辞書を表示します

In [ ]:
# トレーニングデータの'target'列の値のカウントを正規化して表示します
df_train.target.value_counts(normalize=True)  # 各クラスの割合を表示します。これにより、データのバランスを確認できます。

In [ ]:
# クラスの重みを計算します。これにより、クラス間の不均衡を扱います
class_weights = (1 / df_train.target.value_counts(normalize=True).sort_index()).tolist()  # 正規化されたターゲットの値を使って、それらの逆数を計算します
class_weights = torch.tensor(class_weights)  # リストをPyTorchのテンソルに変換します
class_weights = class_weights / class_weights.sum()  # 重みを合計で割って、正規化された重みにします
class_weights  # 計算したクラス重みを表示します

In [ ]:
# トークナイザーを再度設定します。これによりトークン化が可能になります
tokenizer = AutoTokenizer.from_pretrained(  # 基本モデルのトークナイザーを読み込みます
    base_model_id,  # 使用するモデルIDを指定します
    model_max_length=512,  # モデルの最大長を512トークンに設定します
    padding_side="left",  # パディングを左側に設定します
    add_eos_token=True  # 終了トークンを追加する設定をします
)

# トークナイザーのパディングトークンを終了トークンに設定します
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンと同じに設定します

In [ ]:
MAX_LEN = 512  # モデルの最大入力長を512トークンに設定します
col_to_delete = ['summary']  # 削除する列のリストを定義します

def llama_preprocessing_function(examples):  # データを前処理する関数を定義します
    return tokenizer(examples['summary'], truncation=True, max_length=MAX_LEN)  # 'summary'カラムをトークン化し、長さを最大値でトランケーションします

# データセット全体に前処理関数を適用し、バッチ処理を行い、不要な列を削除します
tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)

# 'target'列を'labels'列に名前を変更します
tokenized_datasets = tokenized_datasets.rename_column("target", "labels")

# データセットのフォーマットをPyTorch形式に設定します
tokenized_datasets.set_format("torch")  # トレーニング時にPyTorchが使用できる形式に設定します

In [ ]:
from peft import prepare_model_for_kbit_training  # PEFTからkビットトレーニング用のモデル準備関数をインポートします

model.gradient_checkpointing_enable()  # 勾配チェックポイントを有効にして、メモリを節約します
model = prepare_model_for_kbit_training(model)  # モデルをkビットトレーニング用に準備します

In [ ]:
def print_trainable_parameters(model):  # モデルのトレーニング可能なパラメータを表示する関数
    """
    モデル内のトレーニング可能なパラメータの数を表示します。
    """
    trainable_params = 0  # トレーニング可能なパラメータの初期化
    all_param = 0  # すべてのパラメータの初期化
    for _, param in model.named_parameters():  # モデル内のすべてのパラメータをループします
        all_param += param.numel()  # 総パラメータ数を計算します
        if param.requires_grad:  # パラメータが勾配計算を必要とする場合
            trainable_params += param.numel()  # トレーニング可能なパラメータ数を累積します
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )  # トレーニング可能なパラメータ数、すべてのパラメータ数、トレーニング可能なパラメータの割合を表示します

In [ ]:
# モデルの詳細を表示します
print(model)  # モデルの構造とパラメータの詳細を出力します

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator  # Accelerateライブラリから関連するクラスをインポートします
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig  # FSDPのための設定クラスをインポートします

# Fully Sharded Data Parallelプラグインを設定します
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),  # チェックポイントをCPUにオフロードする設定
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),  # 最適化ステート情報もCPUにオフロードする設定
)

# Acceleratorインスタンスを作成します
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)  # FSDPプラグインを使用してAcceleratorを初期化します

In [ ]:
from peft import LoraConfig, get_peft_model  # PEFTライブラリからLoraConfigとモデル取得関数をインポートします

# LoRAの設定を定義します
config = LoraConfig(
    r=8,  # rankの値
    lora_alpha=16,  # LoRAのスケーリングファクター
    target_modules=[  # 対象モジュールのリスト
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",  # バイアスの設定
    lora_dropout=0.05,  # LoRAのドロップアウト率
    task_type="CAUSAL_LM",  # タスクタイプを指定します（因果言語モデル）
)

# モデルにLoRA設定を適用します
model = get_peft_model(model, config)  # LoRAの設定を持つモデルを取得します
print_trainable_parameters(model)  # トレーニング可能なパラメータを表示します

# アクセラレーターを適用します。必要に応じてコメントアウトできます。
model = accelerator.prepare_model(model)  # Accelerateを使用してモデルを準備します

In [ ]:
# Weights & Biases（WandB）ライブラリをインストールします
!pip install -q wandb -U  # WandBをアップグレードします

import wandb, os  # WandBとOSモジュールをインポートします
wandb.login()  # WandBにログインします

wandb_project = "mistral-finetune"  # プロジェクト名を設定します
if len(wandb_project) > 0:  # プロジェクト名が空でない場合
    os.environ["WANDB_PROJECT"] = wandb_project  # 環境変数にプロジェクト名を設定します

In [ ]:
# GPUの数が1つより多い場合の処理を行います
if torch.cuda.device_count() > 1:  # 使用可能なGPUが1つ以上の場合
    model.is_parallelizable = True  # モデルが並列化可能であることを指定します
    model.model_parallel = True  # モデル並列を有効にします

In [ ]:
# NVIDIAのCUDAコンパイラのバージョンを表示します
!nvcc --version  # nvccのバージョン情報を表示します

# 古いバージョンのPyTorch関連ライブラリをアンインストールします
!pip uninstall torch torchvision torchaudio -y  # PyTorch、TorchVision、およびTorchAudioを強制的にアンインストールします

# CUDA 12.1対応の指定バージョンのPyTorch関連ライブラリをインストールします
!pip install torch==2.0.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.0.0+cu121 -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html  # PyTorchの特定バージョンをインストールします

In [ ]:
# 既存のtorch、torchvision、torchaudioのバージョンをアンインストールします
!pip uninstall torch torchvision torchaudio -y  # PyTorch、TorchVision、およびTorchAudioを強制的にアンインストールします

# CUDA 11.2に対応した互換性のあるバージョンをインストールします
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu112/torch_stable.html  # PyTorchの安定したバージョンをインストールします

In [ ]:
import os  # OSモジュールをインポートします

# CUDAの起動ブロッキングを無効にします
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # CUDAカーネルの起動時にブロッキングを無効にします。これにより、実行時のパフォーマンスが向上する可能性があります。

In [ ]:
import torch  # PyTorchライブラリをインポートします

seed = 7920  # 使用するシード値を設定します

# CUDAが使用可能であれば、CUDAデバイス用の乱数生成器を設定します
if torch.cuda.is_available():
    generator = torch.Generator(device='cuda').manual_seed(seed)  # CUDAデバイス用にシードを設定します
else:
    generator = torch.Generator().manual_seed(seed)  # CPU用にシードを設定します

In [ ]:
# CUDAが利用可能であれば、CUDAデバイス用の乱数生成器を設定します
if torch.cuda.is_available():  
    generator = torch.Generator('cuda').manual_seed(seed)  # CUDAデバイスの乱数生成器をシードで初期化します
else:
    generator = torch.Generator().manual_seed(seed)  # CPU用の乱数生成器をシードで初期化します

In [ ]:
import transformers  # Transformersライブラリをインポートします

class CustomTrainer(transformers.Trainer):  # Trainerクラスを継承したカスタムトレーナークラスを定義します
    def __init__(self, *args, class_weights=None, **kwargs):  # コンストラクタを定義します
        super().__init__(*args, **kwargs)  # 親クラスのコンストラクタを呼び出します
        # クラス重みが与えられた場合、テンソルに変換します
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)  # クラス重みをデバイスに移動します
        else:
            self.class_weights = None  # クラス重みがない場合はNoneとします

    def compute_loss(self, model, inputs, return_outputs=False):  # 損失を計算する関数を定義します
        # ラベルを抽出し、cross_entropy用にlong型に変換します
        labels = inputs.pop("labels").long()  # 入力からラベルを取り出します

        # フォワードパス
        outputs = model(**inputs)  # モデルを介して入力を処理します

        # 出力されたロジットを抽出します
        logits = outputs.get('logits')  # モデル出力からロジットを取り出します

        # 不均衡データ処理のためのクラス重みを用いたカスタム損失を計算します
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)  # クラス重みを使った損失計算
        else:
            loss = F.cross_entropy(logits, labels)  # クラス重みなしで損失計算

        return (loss, outputs) if return_outputs else loss  # 出力の条件に応じて損失と出力を返します

In [ ]:
def compute_metrics(eval_pred):  # 評価予測をもとにメトリクスを計算する関数を定義します
    predictions, labels = eval_pred  # 評価予測から予測値とラベルを取得します
    predictions = np.argmax(predictions, axis=1)  # 予測の中で最大値のインデックスを取得してクラスラベルを決定します
    # 精度とバランス精度を計算して返します
    return {
        'balanced_accuracy': balanced_accuracy_score(predictions, labels),  # バランス精度を計算します
        'accuracy': accuracy_score(predictions, labels)  # 純粋な精度を計算します
    }

In [ ]:
# トレーニングに関する引数を設定します
training_args = transformers.TrainingArguments(
    output_dir='model_classification',  # モデルの出力先ディレクトリ
    learning_rate=1e-4,  # 学習率
    per_device_train_batch_size=8,  # デバイスごとのトレーニングバッチサイズ
    per_device_eval_batch_size=8,  # デバイスごとの評価バッチサイズ
    num_train_epochs=1,  # トレーニングエポック数
    weight_decay=0.01,  # 重み減衰率
    evaluation_strategy='epoch',  # 評価戦略をエポック単位に設定
    save_strategy='epoch',  # モデルの保存戦略をエポック単位に設定
    load_best_model_at_end=True,  # 最良のモデルを最後にロード
	logging_steps=10,  # ロギングの頻度
	max_steps=100,  # 最大ステップ数
    fp16=True,  # 16ビット浮動小数点でトレーニングを実行
    push_to_hub=False  # モデルをHugging Face Hubにプッシュしない設定
)

In [ ]:
# データのコラテーションを行うための関数を設定します
collate_fn = transformers.DataCollatorWithPadding(tokenizer=tokenizer)  # トークナイザーを用いてパディング付きのデータコレータを作成します

In [ ]:
# カスタムトレーナーのインスタンスを作成します
trainer = CustomTrainer(
    model=model,  # 使用するモデル
    args=training_args,  # トレーニング引数
    train_dataset=tokenized_datasets['train'],  # トレーニングデータセット
    tokenizer=tokenizer,  # トークナイザー
    data_collator=collate_fn,  # データコレータ
    compute_metrics=compute_metrics,  # メトリクス計算関数
    class_weights=class_weights,  # クラス重み
)

In [ ]:
# PyTorchライブラリをインストールします
!pip install torch  # 最新のPyTorchをインストールします

In [ ]:
import transformers  # Transformersライブラリをインポートします
from datetime import datetime  # 日付時刻を扱うためのライブラリをインポートします
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # CUDAカーネルの起動をブロッキングします

# CUDAの利用可能性を確認します
print("CUDA Available:", torch.cuda.is_available())  # CUDAが使用可能かどうかを表示します
project = "mistral-finetune"  # プロジェクト名を設定します
base_model_name = "mistral"  # 基本モデル名を設定します
run_name = base_model_name + "-" + project  # 実行名を設定します
output_dir = "./" + run_name  # 出力ディレクトリを設定します

tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンに設定します

# Trainerインスタンスを作成します
trainer = transformers.Trainer(
    model=model,  # 使用するモデル
    train_dataset=tokenized_datasets['train'],  # トレーニングデータセット
    args=transformers.TrainingArguments(
        output_dir=output_dir,  # 出力ディレクトリ
        warmup_steps=5,  # ウォームアップステップ数
        per_device_train_batch_size=2,  # デバイスごとのトレーニングバッチサイズ
        gradient_checkpointing=True,  # 勾配チェックポイントを有効にします
        gradient_accumulation_steps=4,  # 勾配蓄積ステップ数
        max_steps=1000,  # 最大ステップ数
        learning_rate=2.5e-5,  # 学習率
        logging_steps=50,  # ロギングの頻度
        fp16=False,  # 16ビット浮動小数点を無効にします
        optim="paged_adamw_8bit",  # 最適化手法を指定します
        logging_dir="./logs",  # ログの保存先ディレクトリ
        save_strategy="steps",  # ステップごとにモデルチェックポイントを保存
        save_steps=50,  # 50ステップごとにチェックポイントを保存
        evaluation_strategy="steps",  # ステップごとに評価
        eval_steps=50,  # 50ステップごとに評価しチェックポイントを保存
        do_eval=True,  # トレーニングの終了時に評価を実行
        report_to="wandb",  # Weights & Biasesにレポートするかどうか
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",  # W&Bランの名前の設定
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),  # 言語モデル用のデータコラレータを設定
)

model.config.use_cache = False  # 警告を抑制します。推論時には再度有効にしてください
trainer.train()  # トレーニングを開始します

In [ ]:
import torch  # PyTorchライブラリをインポートします
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # 必要なクラスをインポートします

base_model_id = "mistralai/Mistral-7B-v0.1"  # 使用する基本モデルのIDを指定します
bnb_config = BitsAndBytesConfig(  # 量子化に関する設定を作成します
    load_in_4bit=True,  # 4ビットでモデルを読み込む設定
    bnb_4bit_use_double_quant=True,  # 2重量子化を使用する設定
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを指定します（nf4）
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算データ型をbfloat16に設定
)

# 事前学習済みのモデルを読み込みます
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistralモデル
    quantization_config=bnb_config,  # 量子化設定
    device_map="auto",  # 自動デバイスマッピング
    trust_remote_code=True,  # リモートコードを信頼する設定
)

# 評価用のトークナイザーを読み込みます
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,  # 使用するモデルIDを指定します
    add_bos_token=True,  # 始まりのトークンを追加する設定
    trust_remote_code=True,  # リモートコードを信頼する設定
)

In [ ]:
from peft import PeftModel  # PEFTライブラリからPeftModelをインポートします

# 事前学習済みのモデルにファインチューニングされたモデルをロードします
ft_model = PeftModel.from_pretrained(base_model, "mistral-viggo-finetune/checkpoint-1000")  # 指定されたチェックポイントからファインチューニングモデルを読み込みます

In [ ]:
# 評価用のプロンプトを定義します
eval_prompt = """Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

### Target sentence:
Earlier, you stated that you didn't have strong feelings about PlayStation's Little Big Adventure. Is your opinion true for all games which don't have multiplayer?

### Meaning representation:
""" 

# トークナイザーを使用してモデル入力を準備します
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")  # CUDAデバイスに転送します

# モデルを評価モードに設定します
ft_model.eval()
with torch.no_grad():  # 勾配計算を無効にします
    # モデルを使って生成した出力を表示します
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))  # 生成されたトークンをデコードし、特殊トークンをスキップして表示します